In [ ]:
import logging
import random
from collections.abc import Callable
from copy import deepcopy
from dataclasses import dataclass
from datetime import datetime
from json import dumps
from time import time
from typing import Dict, Final, Literal

import numpy as np
import structlog
import torch
import torch.nn as nn
from numpy import ndarray, where
from pandas import DataFrame, Series, concat, read_csv, set_option
from scipy.stats import loguniform, randint, uniform
from sklearn.ensemble import IsolationForest
from sklearn.metrics import (
	auc,
	average_precision_score,
	f1_score,
	precision_recall_curve,
)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from torch.utils.data import DataLoader, Dataset

set_option("display.max_columns", None)
structlog.configure(
	processors=[
		structlog.stdlib.filter_by_level,
		structlog.stdlib.add_logger_name,
		structlog.stdlib.add_log_level,
		structlog.stdlib.PositionalArgumentsFormatter(),
		structlog.processors.TimeStamper(fmt="iso"),
		structlog.processors.StackInfoRenderer(),
		structlog.processors.format_exc_info,
		structlog.processors.UnicodeDecoder(),
		structlog.processors.JSONRenderer(),
	],
	context_class=dict,
	logger_factory=structlog.stdlib.LoggerFactory(),
	wrapper_class=structlog.stdlib.BoundLogger,
	cache_logger_on_first_use=True,
)

type ParamGrid = dict[str, tuple[float | str, ...]]
NUM_TRIALS: Final[int] = 20


@dataclass
class SearchResult:
	"""Results from a hyperparameter search method."""

	method: str
	best_params: dict
	best_score: float
	cv_scores: list[float]
	fit_time: float
	n_evaluations: int

In [ ]:
class SimulatedAnnealingSearch:
	"""
	Custom Simulated Annealing implementation for hyperparameter optimization.

	Simulated Annealing Search:
	- Temperature-based acceptance: Accepts worse solutions with decreasing probability
	- Adaptive parameter perturbation: Different strategies for continuous vs discrete parameters
	- Cooling schedule: Exponential cooling with configurable rate
	- Neighbor generation: Smart parameter space exploration
	"""

	def __init__(
		self,
		param_space: dict,
		n_iter: int = 100,
		initial_temp: float = 1.0,
		cooling_rate: float = 0.95,
		min_temp: float = 0.01,
		random_state: int = 42,
	) -> None:
		"""
		Initialize Simulated Annealing search.

		Args:
			param_space: Dictionary of parameter distributions
			n_iter: Number of iterations
			initial_temp: Initial temperature
			cooling_rate: Rate of temperature decrease
			min_temp: Minimum temperature
			random_state: Random seed
		"""
		self.param_space = param_space
		self.n_iter = n_iter
		self.initial_temp = initial_temp
		self.cooling_rate = cooling_rate
		self.min_temp = min_temp
		self.random_state = random_state
		self.best_params_ = None
		self.best_score_ = -np.inf
		self.cv_results_ = {"mean_test_score": []}

	def _sample_params(self) -> dict:
		"""Sample random parameters from the parameter space."""
		return {
			key: values.rvs(random_state=self.random_state)
			if hasattr(values, "rvs")  # scipy distribution
			else random.choice(values)
			for key, values in self.param_space.items()
		}

	def _neighbor_params(self, current_params: dict) -> dict:
		"""Generate neighboring parameters by slightly modifying current ones."""
		neighbor = deepcopy(current_params)

		# Choose a random parameter to modify
		param_to_modify = random.choice(list(self.param_space.keys()))

		if hasattr(self.param_space[param_to_modify], "rvs"):  # continuous parameter
			if param_to_modify in ["nu", "ocsvm_nu"]:
				# For nu, stay within bounds [0.001, 1.0]
				current_val = neighbor[param_to_modify]
				neighbor[param_to_modify] = np.clip(
					current_val + np.random.normal(0, 0.05 * current_val), 0.001, 1.0
				)
			elif "gamma" in param_to_modify:
				# For gamma, use log-space perturbation
				current_val = neighbor[param_to_modify]
				if isinstance(current_val, (int, float)):
					neighbor[param_to_modify] = 10 ** np.clip(
						np.log10(current_val) + np.random.normal(0, 0.1), -4, 1
					)
			elif "tol" in param_to_modify:
				# For tolerance, use log-space perturbation
				current_val = neighbor[param_to_modify]
				neighbor[param_to_modify] = 10 ** np.clip(
					np.log10(current_val) + np.random.normal(0, 0.1), -6, -1
				)
			else:
				# Generic continuous parameter perturbation
				neighbor[param_to_modify] = self.param_space[param_to_modify].rvs(
					random_state=self.random_state
				)
		else:  # discrete parameter
			neighbor[param_to_modify] = random.choice(self.param_space[param_to_modify])

		return neighbor

	def _evaluate_params(self, params: ParamGrid, X: DataFrame, y: Series) -> float:
		"""Evaluate parameter configuration using cross-validation."""
		return np.mean(
			cross_val_score(
				OneClassSVM(**params, kernel="rbf"), X, y, cv=4, scoring=score_function
			)
		)

	def fit(self, X: DataFrame, y: Series) -> "SimulatedAnnealingSearch":
		"""Fit the simulated annealing search."""
		random.seed(self.random_state)
		np.random.seed(self.random_state)

		# Initialize with random parameters
		current_params = self._sample_params()
		current_score = self._evaluate_params(current_params, X, y)

		self.best_params_ = deepcopy(current_params)
		self.best_score_ = current_score

		temperature = self.initial_temp

		for iteration in range(self.n_iter):
			# Generate neighbor, store the score for cv_results and Accept or reject neighbor
			neighbor_params = self._neighbor_params(current_params)
			neighbor_score = self._evaluate_params(neighbor_params, X, y)
			self.cv_results_["mean_test_score"].append(neighbor_score)

			if neighbor_score > current_score:  # Better solution - always accept
				current_params = neighbor_params
				current_score = neighbor_score
			else:  # Worse solution - accept with probability
				if (
					random.random()
					< np.exp(neighbor_score - current_score / temperature)
					if temperature > 0
					else 0
				):
					current_params = neighbor_params
					current_score = neighbor_score

			if current_score > self.best_score_:  # Update best solution
				self.best_params_ = deepcopy(current_params)
				self.best_score_ = current_score

			# Cool down
			temperature = max(temperature * self.cooling_rate, self.min_temp)

		return self


class GeneticAlgorithmSearch:
	"""
	Custom Genetic Algorithm implementation for hyperparameter optimization.

	Genetic Algorithm Search:
	- Population-based optimization: Maintains diverse parameter sets
	- Tournament selection: Robust parent selection mechanism
	- Uniform crossover: Parameter exchange between parents
	- Adaptive mutation: Random parameter changes with configurable rate
	- Elite preservation: Keeps best solutions across generations
	"""

	def __init__(
		self,
		param_space: dict,
		population_size: int = 20,
		n_generations: int = 10,
		mutation_rate: float = 0.1,
		crossover_rate: float = 0.8,
		elite_size: int = 2,
		random_state: int = 42,
	) -> None:
		"""
		Initialize Genetic Algorithm search.

		Args:
			param_space: Dictionary of parameter distributions
			population_size: Size of population
			n_generations: Number of generations
			mutation_rate: Probability of mutation
			crossover_rate: Probability of crossover
			elite_size: Number of elite individuals to preserve
			random_state: Random seed
		"""
		self.param_space = param_space
		self.population_size = population_size
		self.n_generations = n_generations
		self.mutation_rate = mutation_rate
		self.crossover_rate = crossover_rate
		self.elite_size = elite_size
		self.random_state = random_state
		self.best_params_ = None
		self.best_score_ = -np.inf
		self.cv_results_ = {"mean_test_score": []}

	def _create_individual(self) -> dict:
		"""Create a random individual (parameter set)."""
		return {
			key: values.rvs(random_state=self.random_state)
			if hasattr(values, "rvs")  # scipy distribution
			else random.choice(values)
			for key, values in self.param_space.items()
		}

	def _crossover(self, parent1: dict, parent2: dict) -> Tuple[dict, dict]:
		"""Create two offspring from two parents using uniform crossover."""
		child1, child2 = deepcopy(parent1), deepcopy(parent2)

		for key in parent1.keys():
			if random.random() < 0.5:  # Swap parameter values
				child1[key], child2[key] = child2[key], child1[key]

		return child1, child2

	def _mutate(self, individual: dict) -> dict:
		"""Mutate an individual by randomly changing some parameters."""
		mutated = deepcopy(individual)

		for key in individual.keys():
			if random.random() < self.mutation_rate:
				mutated[key] = (
					self.param_space[key].rvs(random_state=self.random_state)
					if hasattr(self.param_space[key], "rvs")  # continuous parameter
					else random.choice(self.param_space[key])  # discrete parameter
				)
		return mutated

	def _tournament_selection(
		self, population: list, fitness_scores: list, tournament_size: int = 3
	) -> dict:
		"""Select an individual using tournament selection."""
		tournament_indices = random.sample(
			range(len(population)), min(tournament_size, len(population))
		)
		return population[
			tournament_indices[
				np.argmax([fitness_scores[i] for i in tournament_indices])
			]
		]

	def _evaluate_params(self, params: dict, X: DataFrame, y: Series) -> float:
		"""Evaluate parameter configuration using cross-validation."""
		return np.mean(
			cross_val_score(
				OneClassSVM(**params, kernel="rbf"), X, y, cv=4, scoring=score_function
			)
		)

	def fit(self, X: DataFrame, y: Series) -> "GeneticAlgorithmSearch":
		"""Fit the genetic algorithm search."""
		random.seed(self.random_state)
		np.random.seed(self.random_state)
		population = [self._create_individual() for _ in range(self.population_size)]

		for generation in range(self.n_generations):  # Evaluate fitness
			print(f"Evaluating Generation {generation}")
			fitness_scores = []
			for individual in population:
				score = self._evaluate_params(individual, X, y)
				fitness_scores.append(score)
				self.cv_results_["mean_test_score"].append(score)

				if score > self.best_score_:
					self.best_params_ = deepcopy(individual)
					self.best_score_ = score

			# Create next generation | Elite selection - keep best individuals
			new_population = [
				deepcopy(population[idx])
				for idx in np.argsort(fitness_scores)[-self.elite_size :]
			]
			# Generate offspring
			while len(new_population) < self.population_size:
				# Selection
				parent1 = self._tournament_selection(population, fitness_scores)
				parent2 = self._tournament_selection(population, fitness_scores)
				# Crossover
				if random.random() < self.crossover_rate:
					child1, child2 = self._crossover(parent1, parent2)
				else:
					child1, child2 = deepcopy(parent1), deepcopy(parent2)
				# Mutation
				new_population.extend([self._mutate(child1), self._mutate(child2)])
			# Trim to exact population size
			population = new_population[: self.population_size]

		return self

In [ ]:
class LSTMAutoencoder(nn.Module):
	"""
	LSTM-based Autoencoder for time-series reconstruction.

	This model learns to reconstruct normal activity patterns. Anomalies/novelties
	will have higher reconstruction errors.
	"""

	def __init__(
		self,
		input_dim: int,
		hidden_dim: int = 64,
		num_layers: int = 2,
		dropout: float = 0.2,
		bidirectional: bool = False,
	) -> None:
		"""
		Initialize LSTM Autoencoder.

		Args:
			input_dim: Number of input features (sensor channels)
			hidden_dim: Hidden state dimension
			num_layers: Number of LSTM layers
			dropout: Dropout rate between LSTM layers
			bidirectional: Whether to use bidirectional LSTM
		"""
		super(LSTMAutoencoder, self).__init__()

		self.input_dim = input_dim
		self.hidden_dim = hidden_dim
		self.num_layers = num_layers
		self.bidirectional = bidirectional
		self.encoder = nn.LSTM(
			input_size=input_dim,
			hidden_size=hidden_dim,
			num_layers=num_layers,
			batch_first=True,
			dropout=dropout if num_layers > 1 else 0,
			bidirectional=bidirectional,
		)
		# Bottleneck dimension
		encoder_output_dim = hidden_dim * (2 if bidirectional else 1)
		self.decoder = nn.LSTM(
			input_size=encoder_output_dim,
			hidden_size=hidden_dim,
			num_layers=num_layers,
			batch_first=True,
			dropout=dropout if num_layers > 1 else 0,
		)
		self.output_layer = nn.Linear(hidden_dim, input_dim)

	def forward(self, x: torch.Tensor) -> torch.Tensor:
		"""
		Forward pass through the autoencoder.

		Args:
			x: Input tensor of shape (batch, seq_len, input_dim)

		Returns:
			Reconstructed tensor of shape (batch, seq_len, input_dim)
		"""
		batch_size, seq_len, _ = x.shape
		encoded, (hidden, cell) = self.encoder(x)
		# Use only forward direction hidden states if bidirectional
		if self.bidirectional:
			hidden = hidden[: self.num_layers]
			cell = cell[: self.num_layers]

		# Prepare decoder input (repeat last encoded state)
		decoded, _ = self.decoder(
			encoded[:, -1:, :].repeat(1, seq_len, 1), (hidden, cell)
		)
		# Project to original dimension
		return self.output_layer(decoded)


class TimeSeriesDataset(Dataset):
	"""PyTorch Dataset for windowed time-series data."""

	def __init__(self, data: np.ndarray) -> None:
		"""
		Initialize dataset.

		Args:
			data: numpy array of shape (n_windows, window_size, n_features)
		"""
		self.data = torch.FloatTensor(data)

	def __len__(self) -> int:
		"""Return number of windows."""
		return len(self.data)

	def __getitem__(self, idx: int) -> torch.Tensor:
		"""Get a window by index."""
		return self.data[idx]


class LSTMAutoencoderWrapper:
	"""
	Wrapper to make LSTM Autoencoder compatible with sklearn API.

	This wrapper handles training, prediction, and decision function
	computation for the LSTM autoencoder.
	"""

	def __init__(
		self,
		input_dim: int,
		hidden_dim: int = 64,
		num_layers: int = 2,
		dropout: float = 0.2,
		bidirectional: bool = False,
		learning_rate: float = 0.001,
		epochs: int = 50,
		batch_size: int = 64,
		device: str = "cuda" if torch.cuda.is_available() else "cpu",
		threshold_percentile: float = 95.0,
	) -> None:
		"""
		Initialize LSTM Autoencoder wrapper.

		Args:
			input_dim: Number of input features
			hidden_dim: Hidden dimension size
			num_layers: Number of LSTM layers
			dropout: Dropout rate
			bidirectional: Use bidirectional LSTM
			learning_rate: Learning rate for Adam optimizer
			epochs: Number of training epochs
			batch_size: Batch size for training
			device: Device to train on ('cuda' or 'cpu')
			threshold_percentile: Percentile for anomaly threshold
		"""
		self.input_dim = input_dim
		self.hidden_dim = hidden_dim
		self.num_layers = num_layers
		self.dropout = dropout
		self.bidirectional = bidirectional
		self.learning_rate = learning_rate
		self.epochs = epochs
		self.batch_size = batch_size
		self.device = device
		self.threshold_percentile = threshold_percentile

		self.model: LSTMAutoencoder | None = None
		self.threshold: float | None = None

	def fit(self, X: np.ndarray, y: Series | None = None) -> "LSTMAutoencoderWrapper":
		"""
		Train the autoencoder on normal data.

		Args:
			X: numpy array of shape (n_windows, window_size, n_features)
			y: Ignored, for sklearn compatibility

		Returns:
			Self
		"""
		self.model = LSTMAutoencoder(
			input_dim=self.input_dim,
			hidden_dim=self.hidden_dim,
			num_layers=self.num_layers,
			dropout=self.dropout,
			bidirectional=self.bidirectional,
		).to(self.device)
		dataloader = DataLoader(
			TimeSeriesDataset(X),
			batch_size=self.batch_size,
			shuffle=True,
			drop_last=False,
		)
		criterion = nn.MSELoss()  # setup
		optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

		self.model.train()
		for epoch in range(self.epochs):
			print(f"Epoch {epoch + 1}/{self.epochs}")
			epoch_loss = 0.0
			for batch in dataloader:
				batch = batch.to(self.device)
				# Forward pass
				loss = criterion(self.model(batch), batch)
				# Backward pass
				optimizer.zero_grad()
				loss.backward()
				optimizer.step()

				epoch_loss += loss.item()

		# Calculate threshold on training data
		train_scores = self.decision_function(X)
		self.threshold = np.percentile(train_scores, self.threshold_percentile)  # type: ignore

		return self

	def decision_function(self, X: np.ndarray) -> np.ndarray:
		"""
		Calculate reconstruction error (anomaly score).

		Higher values indicate more anomalous samples.

		Args:
			X: numpy array of shape (n_windows, window_size, n_features)

		Returns:
			Anomaly scores for each window
		"""
		if self.model:
			self.model.eval()
		else:
			raise ValueError("Model has not been trained yet. Call fit() first.")

		scores = []
		with torch.no_grad():
			for batch in DataLoader(
				TimeSeriesDataset(X), batch_size=self.batch_size, shuffle=False
			):
				batch = batch.to(self.device)
				scores.append(  # Calculate reconstruction error per sample
					torch.mean((batch - self.model(batch)) ** 2, dim=(1, 2))
					.cpu()
					.numpy()
				)
		return np.concatenate(scores)

	def predict(self, X: np.ndarray) -> np.ndarray:
		"""
		Predict novelty: -1 for anomalies, 1 for normal.

		Args:
			X: numpy array of shape (n_windows, window_size, n_features)

		Returns:
			Predictions: -1 (anomaly) or 1 (normal)
		"""
		return np.where(self.decision_function(X) > self.threshold, -1, 1)

	def get_params(self, deep: bool = True) -> Dict:
		"""Get parameters for sklearn compatibility."""
		return {
			"input_dim": self.input_dim,
			"hidden_dim": self.hidden_dim,
			"num_layers": self.num_layers,
			"dropout": self.dropout,
			"bidirectional": self.bidirectional,
			"learning_rate": self.learning_rate,
			"epochs": self.epochs,
			"batch_size": self.batch_size,
			"threshold_percentile": self.threshold_percentile,
		}

	def set_params(self, **params) -> "LSTMAutoencoderWrapper":
		"""Set parameters for sklearn compatibility."""
		for key, value in params.items():
			setattr(self, key, value)
		return self

In [ ]:
class NoveltyDetectionEnsemble:
	"""
	Ensemble of LSTM Autoencoder, Isolation Forest, and One-Class SVM.

	This ensemble combines three different novelty detection approaches:
	1. LSTM Autoencoder: Captures temporal patterns
	2. Isolation Forest: Fast tree-based anomaly detection
	3. One-Class SVM: Kernel-based boundary learning

	Predictions are combined using weighted voting or averaging of decision scores.
	"""

	def __init__(
		self,
		# Data processing params
		window_size: int = 100,
		stride: int = 50,
		# LSTM params
		lstm_hidden_dim: int = 64,
		lstm_num_layers: int = 2,
		lstm_dropout: float = 0.2,
		lstm_learning_rate: float = 0.001,
		lstm_epochs: int = 30,
		lstm_batch_size: int = 64,
		lstm_threshold_percentile: float = 95.0,
		# Isolation Forest params
		if_n_estimators: int = 100,
		if_contamination: float = 0.1,
		if_max_samples: str | int = "auto",
		# One-Class SVM params
		ocsvm_kernel: str = "rbf",
		ocsvm_nu: float = 0.1,
		ocsvm_gamma: str | float = "scale",
		ocsvm_tol: float = 1e-3,
		# Ensemble params
		weights: Dict[str, float] | None = None,
		voting: str = "soft",  # 'soft' (decision scores) or 'hard' (predictions)
		use_lstm: bool = True,
		use_if: bool = True,
		use_ocsvm: bool = True,
	) -> None:
		"""
		Initialize the ensemble model.

		Args:
			window_size: Size of sliding window for time-series
			stride: Stride for sliding window
			lstm_hidden_dim: LSTM hidden dimension
			lstm_num_layers: Number of LSTM layers
			lstm_dropout: LSTM dropout rate
			lstm_learning_rate: LSTM learning rate
			lstm_epochs: Number of LSTM training epochs
			lstm_batch_size: LSTM batch size
			lstm_threshold_percentile: Percentile for LSTM anomaly threshold
			if_n_estimators: Number of trees in Isolation Forest
			if_contamination: Expected proportion of outliers
			if_max_samples: Number of samples to draw for each tree
			ocsvm_kernel: Kernel type for SVM
			ocsvm_nu: Nu parameter for SVM
			ocsvm_gamma: Gamma parameter for SVM
			ocsvm_tol: Tolerance for SVM
			weights: Dictionary of weights for each model
			voting: Voting strategy ('soft' or 'hard')
			use_lstm: Whether to use LSTM in ensemble
			use_if: Whether to use Isolation Forest in ensemble
			use_ocsvm: Whether to use One-Class SVM in ensemble
		"""
		self.window_size = window_size
		self.stride = stride
		self.lstm_params = {
			"hidden_dim": lstm_hidden_dim,
			"num_layers": lstm_num_layers,
			"dropout": lstm_dropout,
			"learning_rate": lstm_learning_rate,
			"epochs": lstm_epochs,
			"batch_size": lstm_batch_size,
			"threshold_percentile": lstm_threshold_percentile,
		}
		self.if_params = {
			"n_estimators": if_n_estimators,
			"contamination": if_contamination,
			"max_samples": if_max_samples,
			"random_state": 42,
		}
		self.ocsvm_params = {
			"kernel": ocsvm_kernel,
			"nu": ocsvm_nu,
			"gamma": ocsvm_gamma,
			"tol": ocsvm_tol,
		}
		self.use_lstm = use_lstm
		self.use_if = use_if
		self.use_ocsvm = use_ocsvm

		default_weights = {}
		if use_lstm:
			default_weights["lstm"] = 1.0
		if use_if:
			default_weights["if"] = 1.0
		if use_ocsvm:
			default_weights["ocsvm"] = 1.0

		self.weights = weights or default_weights
		self.voting = voting

		self.lstm_model: LSTMAutoencoderWrapper] = None
		self.if_model: IsolationForest] = None
		self.ocsvm_model: OneClassSVM] = None
		self.scaler = StandardScaler()
		self.input_dim: int] = None

	def _create_sliding_windows(
		self, data: DataFrame, activity_col: str = "activity"
	) -> np.ndarray:
		"""
		Create sliding windows from DataFrame.

		Args:
			data: Input DataFrame with sensor data
			activity_col: Name of activity column to exclude

		Returns:
			Windowed array of shape (n_windows, window_size, n_features)
		"""
		# Get sensor columns (exclude activity if present)
		sensor_cols = [col for col in data.columns if col != activity_col]
		values = data[sensor_cols].values
		windows = [
			values[i : i + self.window_size]
			for i in range(0, len(values) - self.window_size + 1, self.stride)
		]
		return (
			np.array(windows)
			if windows
			else np.array([]).reshape(0, self.window_size, len(sensor_cols))
		)

	def fit(
		self, X: DataFrame, y: Series | None = None
	) -> "NoveltyDetectionEnsemble":
		"""
		Train all models in the ensemble.

		Args:
			X: Training DataFrame with sensor data
			y: Ignored, for sklearn compatibility

		Returns:
			Self
		"""
		# Create windowed data
		if len(X_windowed := self._create_sliding_windows(X)) == 0:
			raise ValueError(
				f"No windows created. Data length: {len(X)}, window_size: {self.window_size}"
			)

		n_windows, window_size, n_features = X_windowed.shape
		self.input_dim = n_features

		if self.use_lstm:# 1. Train LSTM Autoencoder on windowed data
			self.lstm_model = LSTMAutoencoderWrapper(
				input_dim=n_features, **self.lstm_params
			)
			self.lstm_model.fit(X_windowed)

		# 2. Flatten windows for classical models and normalize
		X_scaled = self.scaler.fit_transform(X_windowed.reshape(n_windows, -1))

		if self.use_if:# 3. Train Isolation Forest
			self.if_model = IsolationForest(**self.if_params)
			self.if_model.fit(X_scaled)

		if self.use_ocsvm:# 4. Train One-Class SVM
			self.ocsvm_model = OneClassSVM(**self.ocsvm_params)
			self.ocsvm_model.fit(X_scaled)

		return self

	def decision_function(self, X: DataFrame) -> np.ndarray:
		"""
		Calculate weighted ensemble anomaly scores.

		Args:
			X: Test DataFrame with sensor data

		Returns:
			Anomaly scores (higher = more anomalous)
		"""
		# Create windowed data
		if len(X_windowed := self._create_sliding_windows(X)) == 0:
			return np.array([])

		n_windows = X_windowed.shape[0]

		scores_list = []
		weights_list = []

		# LSTM scores (already positive for anomalies)
		if self.use_lstm and self.lstm_model is not None:
			scores_list.append(self._normalize_scores( self.lstm_model.decision_function(X_windowed)))
			weights_list.append(self.weights.get("lstm", 1.0))

		# Flatten and scale for classical models
		X_scaled = self.scaler.transform(X_windowed.reshape(n_windows, -1))

		# Isolation Forest scores (negate to make higher = more anomalous)
		if self.use_if and self.if_model is not None:
			scores_list.append(self._normalize_scores(-self.if_model.score_samples(X_scaled)))
			weights_list.append(self.weights.get("if", 1.0))

		# One-Class SVM scores (negate to make higher = more anomalous)
		if self.use_ocsvm and self.ocsvm_model is not None:
			scores_list.append(self._normalize_scores(-self.ocsvm_model.decision_function(X_scaled)))
			weights_list.append(self.weights.get("ocsvm", 1.0))

		if not scores_list:# Weighted combination
			raise ValueError("No models enabled in ensemble")

		return sum(w * s for w, s in zip(weights_list, scores_list)) / sum(
			weights_list
		) # type: ignore

	def predict(self, X: DataFrame) -> np.ndarray:
		"""
		Predict novelty using ensemble voting.

		Args:
			X: Test DataFrame with sensor data

		Returns:
			Predictions: -1 (anomaly) or 1 (normal)
		"""
		if len(X_windowed:= self._create_sliding_windows(X)) == 0:
			return np.array([])

		if self.voting == "soft":
			# Use decision scores with a threshold
			scores = self.decision_function(X)
			# percentile Can be tuned
			return np.where(scores > np.percentile(scores, 90), -1, 1)
		else:
			# Hard voting
			X_scaled = self.scaler.transform(X_windowed.reshape( X_windowed.shape[0], -1))
			predictions = []

			if self.use_lstm and self.lstm_model is not None:
				predictions.append(self.lstm_model.predict(X_windowed))
			if self.use_if and self.if_model is not None:
				predictions.append(self.if_model.predict(X_scaled))
			if self.use_ocsvm and self.ocsvm_model is not None:
				predictions.append(self.ocsvm_model.predict(X_scaled))

			if not predictions:
				raise ValueError("No models enabled in ensemble")

			# Majority voting
			return np.sign(np.sum(np.stack(predictions), axis=0))

	def _normalize_scores(self, scores: np.ndarray) -> np.ndarray:
		"""
		Normalize scores to [0, 1] range.

		Args:
			scores: Raw anomaly scores

		Returns:
			Normalized scores
		"""
		if (max_score :=np.max(scores)) - (min_score:=np.min(scores)) == 0:
			return np.zeros_like(scores)
		return (scores - min_score) / (max_score - min_score)

	def get_params(self, deep: bool = True) -> dict[str, int | float | None]:
		"""Get parameters for sklearn compatibility."""
		return {
			"window_size": self.window_size,
			"stride": self.stride,
			"lstm_hidden_dim": self.lstm_params["hidden_dim"],
			"lstm_num_layers": self.lstm_params["num_layers"],
			"lstm_dropout": self.lstm_params["dropout"],
			"lstm_learning_rate": self.lstm_params["learning_rate"],
			"lstm_epochs": self.lstm_params["epochs"],
			"lstm_batch_size": self.lstm_params["batch_size"],
			"lstm_threshold_percentile": self.lstm_params["threshold_percentile"],
			"if_n_estimators": self.if_params["n_estimators"],
			"if_contamination": self.if_params["contamination"],
			"if_max_samples": self.if_params["max_samples"],
			"ocsvm_kernel": self.ocsvm_params["kernel"],
			"ocsvm_nu": self.ocsvm_params["nu"],
			"ocsvm_gamma": self.ocsvm_params["gamma"],
			"ocsvm_tol": self.ocsvm_params["tol"],
			"weights": self.weights,
			"voting": self.voting,
			"use_lstm": self.use_lstm,
			"use_if": self.use_if,
			"use_ocsvm": self.use_ocsvm,
		}

	def set_params(self, **params) -> "NoveltyDetectionEnsemble":
		"""Set parameters for sklearn compatibility."""
		for key, value in params.items():
			if key.startswith("lstm_"):
				param_name = key.replace("lstm_", "")
				if param_name in self.lstm_params:
					self.lstm_params[param_name] = value
			elif key.startswith("if_"):
				param_name = key.replace("if_", "")
				if param_name in self.if_params:
					self.if_params[param_name] = value
			elif key.startswith("ocsvm_"):
				param_name = key.replace("ocsvm_", "")
				if param_name in self.ocsvm_params:
					self.ocsvm_params[param_name] = value
			elif key in [
				"weights",
				"voting",
				"window_size",
				"stride",
				"use_lstm",
				"use_if",
				"use_ocsvm",
			]:
				setattr(self, key, value)
		return self

In [ ]:
def configure_file_logger(filepath: str) -> structlog.BoundLogger:
	"""
	Configure a file-specific logger using structlog.

	Args:
		filepath: Path to log file

	Returns:
		Configured structlog logger
	"""
	# Create a specific handler for this file
	file_handler = logging.FileHandler(filepath, mode="a")
	file_handler.setLevel(logging.INFO)

	# Configure structlog with file output
	structlog.configure(
		processors=[
			structlog.stdlib.filter_by_level,
			structlog.stdlib.add_logger_name,
			structlog.stdlib.add_log_level,
			structlog.stdlib.PositionalArgumentsFormatter(),
			structlog.processors.TimeStamper(fmt="iso"),
			structlog.processors.StackInfoRenderer(),
			structlog.processors.format_exc_info,
			structlog.processors.UnicodeDecoder(),
			structlog.processors.JSONRenderer(),
		],
		context_class=dict,
		logger_factory=structlog.stdlib.LoggerFactory(),
		wrapper_class=structlog.stdlib.BoundLogger,
		cache_logger_on_first_use=False,
	)
	# Get the underlying stdlib logger and add the handler
	stdlib_logger = logging.getLogger("hyperparameter_search")
	stdlib_logger.handlers.clear()  # Clear existing handlers
	stdlib_logger.addHandler(file_handler)
	stdlib_logger.setLevel(logging.INFO)

	return structlog.get_logger("hyperparameter_search")


def score_ensemble_function(
	model: NoveltyDetectionEnsemble, Train: DataFrame, test: Series
) -> float:
	"""
	Objective function to maximize, calculates the F1 score on the test set.
	Follows the format needed by scikit-learn's API.

	Args:
		model: NoveltyDetectionEnsemble to evaluate
		Train: Train data, only for API compliance
		test: True targets, only for API compliance

	Returns:
		F1 score
	"""
	global testing_data, test_targets, logger

	# If no predictions (empty data), return 0
	if len(predictions := model.predict(testing_data)) == 0:
		logger.warning("No predictions generated - empty windowed data")
		return 0.0

	# Align predictions with targets (use first N targets that match windows)
	aligned_targets = test_targets.iloc[: len(predictions)].values

	f1 = f1_score(aligned_targets, predictions == -1)

	# Get decision scores
	if len(anomaly_scores := model.decision_function(testing_data)) > 0:
		precision, recall, _ = precision_recall_curve(aligned_targets, anomaly_scores)
		avg_precision = average_precision_score(aligned_targets, anomaly_scores)
		auc_pr = auc(recall, precision)
	else:
		avg_precision = 0.0
		auc_pr = 0.0

	logger.info(
		{
			"target": f1,
			"avg_precision": avg_precision,
			"auc_pr": auc_pr,
			"params": model.get_params(),
			"n_windows": len(predictions),
			"datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
		}
	)
	return float(f1)

In [ ]:
def get_param_grid(
	search_method: Literal[
		"Grid",
		"Random",
		"SimulatedAnnealing",
		"GeneticAlgorithm",
		"Bayesian",
		"Ensemble",
	],
	use_log_dist: bool = False,
) -> ParamGrid:
	"""
	Get parameter grid for different search methods.

	Args:
		search_method: Name of search method
		use_log_dist: Whether to use log-uniform distributions

	Returns:
		Parameter grid dictionary
	"""
	if search_method == "Ensemble":
		if use_log_dist:
			return {
				# Window params
				"window_size": [50, 100, 150, 200],
				"stride": [25, 50, 75],
				# LSTM params
				"lstm_hidden_dim": [32, 64, 128],
				"lstm_num_layers": [1, 2, 3],
				"lstm_dropout": uniform(0.0, 0.5),
				"lstm_learning_rate": loguniform(1e-4, 1e-2),
				"lstm_epochs": [20, 30, 50],
				# IF params
				"if_n_estimators": [100, 200, 500],
				"if_contamination": uniform(0.01, 0.19),
				# OCSVM params
				"ocsvm_nu": loguniform(0.001, 0.3),
				"ocsvm_gamma": loguniform(1e-4, 10),
				"ocsvm_tol": loguniform(1e-6, 1e-1),
			}
		else:
			return {
				"window_size": [100, 150],
				"stride": [50],
				"lstm_hidden_dim": [64, 128],
				"lstm_num_layers": [2],
				"lstm_dropout": [0.2],
				"lstm_learning_rate": [0.001],
				"lstm_epochs": [30],
				"if_n_estimators": [100],
				"if_contamination": [0.1],
				"ocsvm_nu": [0.01, 0.05, 0.1],
				"ocsvm_gamma": ["scale", "auto"],
				"ocsvm_tol": [1e-3],
			}
	elif search_method == "Grid":
		return {
			"nu": [0.01, 0.05, 0.1, 0.25],
			"gamma": ["scale", "auto", 0.001, 0.01, 0.1],
			"tol": [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
		}
	elif search_method in ["Random", "SimulatedAnnealing", "GeneticAlgorithm"]:
		if use_log_dist:  # Log-uniform for better coverage
			return {
				"nu": loguniform(0.001, 0.3),
				"gamma": loguniform(1e-4, 10),
				"tol": loguniform(1e-6, 1e-1),
			}
		else:
			return {
				"nu": [0.01, 0.025, 0.05, 0.75, 0.1, 0.2, 0.3, 0.4, 0.5],
				"gamma": ["scale", "auto", 0.001, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1],
				"tol": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
			}
	elif search_method == "Bayesian":
		return {"nu": (0.01, 0.5), "gamma": (1e-4, 1), "tol": (1e-5, 1e-1)}

	return {}


def score_function(model: OneClassSVM, Train: DataFrame, test: Series) -> float:
	"""
	Objective function to maximize, calculates the F1 score on the test set.
	Follows the format needed by scikit-learn's API.

	Args:
		model: OneClassSVM to evaluate
		X_test: Train data, only for API compliance
		y_true: True targets, only for API compliance

	Returns:
		F1 score
	"""
	global testing_data, test_targets, logger

	f1 = f1_score(test_targets, where(model.predict(testing_data) == -1, True, False))

	# Get decision scores (higher values = more normal, lower = more anomalous)
	# convert to anomaly scores (higher values = more anomalous) and Negate
	# decision scores since OneClassSVM gives higher scores for inliers
	anomaly_scores = -model.decision_function(testing_data)
	precision, recall, _ = precision_recall_curve(test_targets, anomaly_scores)
	logger.info(
		{
			"target": f1,
			"avg_precision": average_precision_score(test_targets, anomaly_scores),
			"auc_pr": auc(recall, precision),
			"params": model.get_params(),
			"datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
		}
	)
	return float(f1)

In [ ]:
def update_train_vars(
	i: int, activities: ndarray
) -> tuple[DataFrame, Series, DataFrame, Series]:
	"""
	Update training and testing variables for current activity.

	Args:
		i: Current activity index
		activities: Array of all activities

	Returns:
		Tuple of (training_data, train_targets, testing_data, test_targets)
	"""
	global X_train, X_test, MIN_SAMPLES

	training = (  # picks the first n samples of each class
		X_train[X_train["activity"].isin(activities[:i])]
		.groupby("activity")
		.head(MIN_SAMPLES)
	)
	testing = X_test[X_test["activity"] == activities[i]].head(MIN_SAMPLES)
	training.loc[:, "isNovelty"], testing.loc[:, "isNovelty"] = False, True
	novelty = concat(
		[testing, training.sample(n=int(0.15 * len(training)), random_state=42)]
	)
	return (
		training.drop(columns=["isNovelty"]),
		training["isNovelty"],
		# only current activity (as novelty)
		novelty.drop(columns=["isNovelty"]),
		novelty["isNovelty"],
	)


def train_search_method(
	training_data: DataFrame,
	train_targets: Series,
	search_type: Literal[
		"Grid", "Random", "SimulatedAnnealing", "GeneticAlgorithm", "Ensemble"
	],
	params: dict[str, list],
	scoring: Callable,
	n_iter: int | None = 100,
	cv: int = 4,
	verbose: int = 1,
	random_state: int = 42,
) -> (
	RandomizedSearchCV
	| GridSearchCV
	| SimulatedAnnealingSearch
	| GeneticAlgorithmSearch
):
	"""
	Train a model using the specified search method.

	Args:
		training_data: Training DataFrame
		train_targets: Training targets
		search_type: Type of search method
		params: Parameter grid/distributions
		scoring: Scoring function
		n_iter: Number of iterations (for RandomSearch and metaheuristics)
		cv: Number of CV folds
		verbose: Verbosity level
		random_state: Random seed

	Returns:
		Fitted search object
	"""
	if search_type == "SimulatedAnnealing":
		return SimulatedAnnealingSearch(
			param_space=params,
			n_iter=n_iter or 100,
			initial_temp=1.0,
			cooling_rate=0.95,
			random_state=random_state,
		).fit(training_data, train_targets)

	elif search_type == "GeneticAlgorithm":
		# For GA, use n_iter as total evaluations = population_size * n_generations
		population_size = min(20, n_iter // 5) if n_iter else 20
		return GeneticAlgorithmSearch(
			param_space=params,
			population_size=population_size,
			n_generations=(n_iter // population_size) if n_iter else 5,
			mutation_rate=0.1,
			crossover_rate=0.8,
			random_state=random_state,
		).fit(training_data, train_targets)

	else:
		# Original implementation for Grid and Random search
		search_cls = RandomizedSearchCV if search_type == "Random" else GridSearchCV

		# Determine which model to use
		if search_type == "Ensemble":
			base_model = NoveltyDetectionEnsemble()
		else:
			base_model = OneClassSVM(kernel="rbf")

		search_kwargs = {
			f"param_{'distributions' if search_type == 'Random' else 'grid'}": params,
			"estimator": base_model,
			"scoring": scoring,
			"cv": cv,
			"verbose": verbose,
			"error_score": "raise",
		}
		if search_type == "Random" and n_iter:
			search_kwargs.update({"n_iter": n_iter, "random_state": random_state})
		return search_cls(**search_kwargs).fit(training_data, train_targets)

In [ ]:
def compare_search_methods(
	grid_scores: list[float], random_scores: list[float]
) -> Dict[str, float]:
	"""
	Statistical comparison of search methods using Wilcoxon signed-rank test.

	Args:
		grid_scores: Scores from first method
		random_scores: Scores from second method

	Returns:
		Dictionary with comparison statistics
	"""
	if len(grid_scores) != len(random_scores):
		raise ValueError("Score arrays must have equal length")

	statistic, p_value = wilcoxon(grid_scores, random_scores, alternative="two-sided")

	return {
		"wilcoxon_statistic": statistic,
		"p_value": p_value,
		"grid_mean": np.mean(grid_scores),
		"random_mean": np.mean(random_scores),
		"effect_size": (np.mean(grid_scores) - np.mean(random_scores))
		/ np.std(grid_scores + random_scores),
	}


def update_params_grid(
	cv_results: dict[str, tuple], og_param_grid: ParamGrid
) -> ParamGrid:
	"""
	Update parameter grid based on cross-validation results.

	Args:
		cv_results: Cross-validation results
		og_param_grid: Original parameter grid

	Returns:
		Updated parameter grid
	"""
	params = ["gamma", "nu", "tol"]
	top_entries = (
		DataFrame(
			zip(
				cv_results["rank_test_score"],
				cv_results["param_gamma"],
				cv_results["param_nu"],
				cv_results["param_tol"],
			),
			columns=["rank_test_score", "gamma", "nu", "tol"],
		)
		.sort_values("rank_test_score")
		.head(NUM_TRIALS)
	)
	# Check if we're stuck with the same parameter space
	if len(top_entries) == NUM_TRIALS:
		print("Detected potential parameter space stagnation, diversifying...")

		current_params = {col: set(top_entries[col]) for col in params}
		unused_params = {
			param: list(set(og_param_grid[param]) - current_params[param])
			for param in params
		}
		# Replace least effective values with unused ones
		for param in params:
			if unused_params[param]:  # If there are unused values available
				current_unique = list(dict.fromkeys(top_entries[param]))
				# Remove the least effective (last) value and add an unused one
				if len(current_unique) > 1 and unused_params[param]:
					current_unique = current_unique[:-1] + [unused_params[param][0]]
				elif unused_params[param]:
					# If only one value, replace it partially
					current_unique.append(unused_params[param][0])

				top_entries.loc[  # Update the parameter list
					top_entries[param] == list(dict.fromkeys(top_entries[param]))[-1],
					param,
				] = unused_params[param][0]

	exmp = {col: list(dict.fromkeys(top_entries[col])) for col in params}
	cartesian_size = len(exmp["gamma"]) * len(exmp["nu"]) * len(exmp["tol"])

	if cartesian_size > NUM_TRIALS:
		while cartesian_size > NUM_TRIALS:
			# Find which parameter to reduce (try removing last value from each)
			best_reduction = None
			best_param = None

			for param in params:
				if len(exmp[param]) > 1:  # Only reduce if more than 1 value remains
					# Calculate new cartesian size if we remove last value from this param
					temp_sizes = [
						len(exmp[p]) if p != param else len(exmp[p]) - 1 for p in params
					]
					new_size = temp_sizes[0] * temp_sizes[1] * temp_sizes[2]
					# Check if this gets us closer to NUM_TRIALS without going under
					if new_size >= NUM_TRIALS and (
						best_reduction is None or new_size < best_reduction
					):
						best_reduction = new_size
						best_param = param

			# If no good reduction found, just remove from the param with most values
			if best_param is None:
				param_lengths = [(param, len(exmp[param])) for param in params]
				param_lengths.sort(key=lambda x: x[1], reverse=True)
				best_param = param_lengths[0][0]

			# Remove the last (least effective) value from the chosen parameter
			if len(exmp[best_param]) > 1:
				exmp[best_param] = exmp[best_param][:-1]

			cartesian_size = len(exmp["gamma"]) * len(exmp["nu"]) * len(exmp["tol"])
			# Safety break to avoid infinite loop
			if all(len(exmp[param]) == 1 for param in params):
				break

	assert cartesian_size == NUM_TRIALS, "reducing the params space failed"

	print(f"dict of len {cartesian_size} :", exmp)
	return exmp

In [ ]:
def eval_search_method(
	activities: ndarray,
	search_name: Literal[
		"Grid", "Random", "SimulatedAnnealing", "GeneticAlgorithm", "Ensemble"
	],
	use_log_dist: bool = False,
) -> SearchResult:
	"""
	Evaluate a hyperparameter search method across activities.

	Args:
		activities: Array of activity labels
		search_name: Name of search method
		use_log_dist: Whether to use log-uniform distributions

	Returns:
		SearchResult with evaluation metrics
	"""
	dist = get_param_grid(search_name, use_log_dist)
	MAXIMAZED = False
	BEST_SCORES = None

	global testing_data, test_targets, logger
	start_time = time()

	# Determine scoring function based on search type
	scoring_func = (
		score_ensemble_function if search_name == "Ensemble" else score_function
	)

	for i in range(1, len(activities)):
		training_data, train_targets, test_data, testing_targets = update_train_vars(
			i, activities
		)
		testing_data = test_data
		test_targets = testing_targets
		print(f"Training for activities {activities[:i]}")

		if not MAXIMAZED:
			search_method = train_search_method(
				training_data=training_data,
				train_targets=train_targets,
				search_type=search_name,
				params=dist,
				scoring=scoring_func,
			)
			BEST_SCORES = search_method
			MAXIMAZED = True
		else:
			print(f"Already maximized, suggesting new {NUM_TRIALS} points")
			# For metaheuristics and ensemble, we don't update param grid like GridSearch
			if search_name in ["SimulatedAnnealing", "GeneticAlgorithm", "Ensemble"]:
				search_method = train_search_method(
					training_data=training_data,
					train_targets=train_targets,
					search_type=search_name,
					params=dist,
					scoring=scoring_func,
					n_iter=NUM_TRIALS,
				)
			else:
				search_method = train_search_method(
					training_data=training_data,
					train_targets=train_targets,
					search_type=search_name,
					params=update_params_grid(search_method.cv_results_, dist)
					if search_name == "Grid"
					else dist,
					scoring=scoring_func,
					n_iter=NUM_TRIALS if search_name == "Random" else None,
				)

			if search_method.best_score_ > BEST_SCORES.best_score_:
				BEST_SCORES = search_method

		print(f"{search_name} Search Best Params:", search_method.best_params_)

	return SearchResult(
		method=search_name + "_search",
		best_params=BEST_SCORES.best_params_,
		best_score=BEST_SCORES.best_score_,
		cv_scores=BEST_SCORES.cv_results_["mean_test_score"].tolist()
		if hasattr(BEST_SCORES.cv_results_["mean_test_score"], "tolist")
		else BEST_SCORES.cv_results_["mean_test_score"],
		fit_time=time() - start_time,
		n_evaluations=len(BEST_SCORES.cv_results_["mean_test_score"]),
	)

In [ ]:
X_train = read_csv("../data/PAMAP2/x_train_data.csv")
X_test = read_csv("../data/PAMAP2/x_test_data.csv")
y_train = read_csv("../data/PAMAP2/y_train_data.csv")
y_test = read_csv("../data/PAMAP2/y_test_data.csv")

X_train["activity"] = y_train
X_test["activity"] = y_test

testing_data: DataFrame
test_targets: Series

MIN_SAMPLES = X_train["activity"].value_counts().sort_values().iloc[0]
MAXIMAZED = False

activities = X_train["activity"].unique()

# Run original searches
logger = configure_file_logger("../reports/logs_grid.log")
grid_result = eval_search_method(activities, "Grid")

logger = configure_file_logger("../reports/logs_rand.log")
rand_result = eval_search_method(activities, "Random")

logger = configure_file_logger("../reports/logs_rand_log.log")
rand_log_result = eval_search_method(activities, "Random", True)

# New metaheuristic searches
logger = configure_file_logger("../reports/logs_sian.log")
sa_result = eval_search_method(activities, "SimulatedAnnealing", True)

logger = configure_file_logger("../reports/logs_geal.log")
ga_result = eval_search_method(activities, "GeneticAlgorithm", True)

# New ensemble search
logger = configure_file_logger("../reports/logs_ensemble.log")
ensemble_result = eval_search_method(activities, "Ensemble", True)

# Compare all methods
with open("../conf/test_results.json", "w") as file:
	comparisons = [
		{
			"test": "Grid x Rand",
			**compare_search_methods(grid_result.cv_scores, rand_result.cv_scores),
		},
		{
			"test": "Grid x SA",
			**compare_search_methods(grid_result.cv_scores, sa_result.cv_scores),
		},
		{
			"test": "Grid x GA",
			**compare_search_methods(grid_result.cv_scores, ga_result.cv_scores),
		},
		{
			"test": "Grid x Ensemble",
			**compare_search_methods(grid_result.cv_scores, ensemble_result.cv_scores),
		},
		{
			"test": "Rand x SA",
			**compare_search_methods(rand_result.cv_scores, sa_result.cv_scores),
		},
		{
			"test": "Rand x GA",
			**compare_search_methods(rand_result.cv_scores, ga_result.cv_scores),
		},
		{
			"test": "Rand x Ensemble",
			**compare_search_methods(rand_result.cv_scores, ensemble_result.cv_scores),
		},
		{
			"test": "SA x GA",
			**compare_search_methods(sa_result.cv_scores, ga_result.cv_scores),
		},
		{
			"test": "SA x Ensemble",
			**compare_search_methods(sa_result.cv_scores, ensemble_result.cv_scores),
		},
		{
			"test": "GA x Ensemble",
			**compare_search_methods(ga_result.cv_scores, ensemble_result.cv_scores),
		},
	]
	file.write(dumps(comparisons, indent=2))

print("\n" + "=" * 80)
print("FINAL RESULTS SUMMARY")
print("=" * 80)
for result in [
	grid_result,
	rand_result,
	rand_log_result,
	sa_result,
	ga_result,
	ensemble_result,
]:
	print(f"\n{result.method}:")
	print(f"  Best Score: {result.best_score:.4f}")
	print(f"  Mean CV Score: {np.mean(result.cv_scores):.4f}")
	print(f"  Fit Time: {result.fit_time:.2f}s")
	print(f"  N Evaluations: {result.n_evaluations}")